# Stats in Paper

In [1]:
# imports
import ipdb, os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import collections

## IBM

In [2]:
IBM_train_csv = "~/Research/task-dataset-metric-nli-extraction/data/ibm/exp/few-shot-setup/NLP-TDMS/paperVersion/train.tsv"
IBM_test_csv = "~/Research/task-dataset-metric-nli-extraction/data/ibm/exp/few-shot-setup/NLP-TDMS/paperVersion/test.tsv"

In [3]:
train_IBM = pd.read_csv(IBM_train_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])
# train_IBM['label'] = train_IBM.label.apply(lambda x: "true" if x else "false")

test_IBM = pd.read_csv(IBM_test_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])

In [4]:
def get_stats(path_to_train_df, path_to_test_df): 
    
    TDM_over_all = set()
    unique_labels = path_to_train_df[(path_to_train_df.label == True)].TDM.tolist()
    
    TDM = set()
    Uniq_task = set()
    Uniq_dataset = set()
    Uniq_metric = set()
    unknown_count = 0
    avg_tdm_per_paper = defaultdict(lambda : 0)
    
    TDM_count = defaultdict(lambda : 0)
    
    for contrib in unique_labels:
        split = contrib.split(';')
        
        if(len(split) == 1):
            
            unknown_count += 1 
        else:
            if len(split) !=3:
#                 ipdb.set_trace()
                task, dataset, metric, _ = split
                
            else:
                task, dataset, metric = split
            
            t, d, m = task.strip(), dataset.strip(), metric.strip()
            TDM.add(f"{t}#{d}#{m}")
            TDM_over_all.add(f"{t}#{d}#{m}")
            
            TDM_count[f"{t}#{d}#{m}"] += 1
            
            Uniq_task.add(t)
            Uniq_dataset.add(d)
            Uniq_metric.add(m)
    
    for paper in path_to_train_df[(path_to_train_df.label == True) & (path_to_train_df.TDM != 'unknown') ].title.tolist():
        avg_tdm_per_paper[paper] += 1
    
    print("Train: ")
    print(f"Number of papers: {len(set(path_to_train_df[(path_to_train_df.label == True)].title.tolist()))}")
    print(f"Unknown count: {unknown_count}")
    print(f"Total leaderboards: {len(path_to_train_df[(path_to_train_df.label == True) & (path_to_train_df.TDM != 'unknown')].title.tolist())}")
    print(f"Avg leaderboard per paper: {round(np.mean(list(avg_tdm_per_paper.values())), 2)}")
    print(f"Distinc leaderboard: {len(TDM)}")
    print(f"Distinct taks: {len(Uniq_task)}")
    print(f"Distinc datasets: {len(Uniq_dataset)}")
    print(f"Distinc metrics: {len(Uniq_metric)}")
    print(f"Max leaderboard per paper: {round(np.max(list(avg_tdm_per_paper.values())), 2)}")
    print(f"Min leaderboard per paper: {round(np.min(list(avg_tdm_per_paper.values())), 2)}")
    
    oder_TDM_count1 = sorted(TDM_count.items(), key=lambda item: item[1])
    print(f"Least frequent leaderboard :{oder_TDM_count1[:3]}")
    print(f"Most frequent leaderboard :{oder_TDM_count1[-3:]}")
    
    print(" \n==========================================================================\n ")
    unique_labels = path_to_test_df[(path_to_test_df.label == True)].TDM.tolist()
    
    TDM = set()
    Uniq_task = set()
    Uniq_dataset = set()
    Uniq_metric = set()
    unknown_count = 0
    avg_tdm_per_paper = defaultdict(lambda : 0)
    
    TDM_count = defaultdict(lambda : 0)
    
    for contrib in unique_labels:
        split = contrib.split(';')
        
        if(len(split) == 1):
            
            unknown_count += 1 
        else:
            if len(split) !=3:
#                 ipdb.set_trace()
                task, dataset, metric, _ = split
                
            else:
                task, dataset, metric = split
            
            t, d, m = task.strip(), dataset.strip(), metric.strip()
            TDM.add(f"{t}#{d}#{m}")
            TDM_over_all.add(f"{t}#{d}#{m}")
            
            TDM_count[f"{t}#{d}#{m}"] += 1
            
            Uniq_task.add(t)
            Uniq_dataset.add(d)
            Uniq_metric.add(m)
    
    for paper in path_to_test_df[(path_to_test_df.label == True) & (path_to_test_df.TDM != 'unknown') ].title.tolist():
        avg_tdm_per_paper[paper] += 1
    print("Test: ")
    print(f"Number of papers: {len(set(path_to_test_df[(path_to_test_df.label == True)].title.tolist()))}")
    print(f"Unknown count: {unknown_count}")
    print(f"Total leaderboards: {len(path_to_test_df[(path_to_test_df.label == True) & (path_to_test_df.TDM != 'unknown')].title.tolist())}")
    print(f"Avg leaderboard per paper: {round(np.mean(list(avg_tdm_per_paper.values())), 2)}")
    print(f"Distinc leaderboard: {len(TDM)}")
    print(f"Distinct taks: {len(Uniq_task)}")
    print(f"Distinc datasets: {len(Uniq_dataset)}")
    print(f"Distinc metrics: {len(Uniq_metric)}")
    print(f"Max leaderboard per paper: {round(np.max(list(avg_tdm_per_paper.values())), 2)}")
    print(f"Min leaderboard per paper: {round(np.min(list(avg_tdm_per_paper.values())), 2)}")
    
    oder_TDM_count2 = sorted(TDM_count.items(), key=lambda item: item[1])
    print(f"Least frequent leaderboard :{oder_TDM_count2[:3]}")
    print(f"Most frequent leaderboard :{oder_TDM_count2[-3:]}")
    
    print(" \n==========================================================================\n ")
    
    print(f"Unique Triples over all: {len(TDM_over_all)}")
    
    return oder_TDM_count1

In [5]:
def get_stats_old(path_to_df): 
    
    unique_labels = path_to_df[(path_to_df.label == True)].TDM.tolist()
    
    TDM = set()
    Uniq_task = set()
    Uniq_dataset = set()
    Uniq_metric = set()
    unknown_count = 0
    avg_tdm_per_paper = defaultdict(lambda : 0)
    
    for contrib in unique_labels:
        split = contrib.split(';')
        
        if(len(split) == 1):
            
            unknown_count += 1 
        else:
            if len(split) !=3:
#                 ipdb.set_trace()
                task, dataset, metric, _ = split
                
            else:
                task, dataset, metric = split
            
            t, d, m = task.strip(), dataset.strip(), metric.strip()
            TDM.add(f"{t}#{d}#{m}")
            
            Uniq_task.add(t)
            Uniq_dataset.add(d)
            Uniq_metric.add(m)
    
    for paper in path_to_df[(path_to_df.label == True) & (path_to_df.TDM != 'unknown') ].title.tolist():
        avg_tdm_per_paper[paper] += 1
    
    print(f"Number of papers: {len(set(path_to_df[(path_to_df.label == True)].title.tolist()))}")
    print(f"Unknown count: {unknown_count}")
    print(f"Total leaderboards: {len(path_to_df[(path_to_df.label == True) & (path_to_df.TDM != 'unknown')].title.tolist())}")
    print(f"Avg leaderboard per paper: {round(np.mean(list(avg_tdm_per_paper.values())), 2)}")
    print(f"Distinc leaderboard: {len(TDM)}")
    print(f"Distinct taks: {len(Uniq_task)}")
    print(f"Distinc datasets: {len(Uniq_dataset)}")
    print(f"Distinc metrics: {len(Uniq_metric)}")
    print(f"Max leaderboard per paper: {round(np.max(list(avg_tdm_per_paper.values())), 2)}")
    print(f"Min leaderboard per paper: {round(np.min(list(avg_tdm_per_paper.values())), 2)}")
    
    
    return avg_tdm_per_paper

### Train

In [6]:
train_IBM.head()

,label,title,TDM,Context
0,True,D16-1036.pdf,unknown,Multi-view Response Selection for Human-Comput...
1,False,D16-1036.pdf,question answering; SQuAD; F1,Multi-view Response Selection for Human-Comput...
2,False,D16-1036.pdf,relation prediction; FB15K-237; H@1,Multi-view Response Selection for Human-Comput...
3,False,D16-1036.pdf,word sense disambiguation; SemEval 2013; F1,Multi-view Response Selection for Human-Comput...
4,False,D16-1036.pdf,language modeling; 1B Words / Google Billion W...,Multi-view Response Selection for Human-Comput...


In [7]:
train_IBM.title.nunique()

170

In [8]:
# avg_tdm_per_paper = get_stats(train_IBM)

In [9]:
# avg_tdm_per_paper = get_stats(test_IBM)

In [10]:
avg_tdm_per_paper = get_stats(train_IBM, test_IBM)

Train: 
Number of papers: 170
Unknown count: 46
Total leaderboards: 327
Avg leaderboard per paper: 2.64
Distinc leaderboard: 78
Distinct taks: 18
Distinc datasets: 44
Distinc metrics: 31
Max leaderboard per paper: 10
Min leaderboard per paper: 1
Least frequent leaderboard :[('language modeling#Penn Treebank#Bit per Character (BPC)', 1), ('chunking#Penn Treebank#F1', 2), ('word segmentation#MSR#F1', 2)]
Most frequent leaderboard :[('dependency parsing#Penn Treebank#UAS', 8), ('question answering#SQuAD#F1', 11), ('question answering#SQuAD#EM', 11)]
 
 
Test: 
Number of papers: 167
Unknown count: 45
Total leaderboards: 294
Avg leaderboard per paper: 2.41
Distinc leaderboard: 78
Distinct taks: 18
Distinc datasets: 44
Distinc metrics: 31
Max leaderboard per paper: 7
Min leaderboard per paper: 1
Least frequent leaderboard :[('sentiment analysis#SUBJ#Accuracy', 2), ('text classification#DBpedia#Error', 2), ('text classification#AG News#Error', 2)]
Most frequent leaderboard :[('dependency pars

In [11]:
# Make sure that all leaderboard in test are present in train 
count = []
for paper in test_IBM.TDM.to_list():
    if paper not in train_IBM.TDM.to_list():
        count.append(paper)
print(count)

[]


In [12]:
count = []
for paper in train_IBM.TDM.to_list():
    if paper not in test_IBM.TDM.to_list():
        print(paper)
        count.append(paper)
print(count)

[]


### our dataset

In [52]:
# New_train_csv = "~/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_800/twofoldwithunk/fold1/train.tsv"
# New_test_csv = "~/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_800/twofoldwithunk/fold1/dev.tsv"

# New_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/twofoldwithunk/fold2/train.tsv"
# New_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_10000/10Neg10000unk/twofoldwithunk/fold2/dev.tsv"

# New_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_5000/10Neg5000unk/twofoldwithunk/fold2/train.tsv"
# New_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_5000/10Neg5000unk/twofoldwithunk/fold2/dev.tsv"

# New_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_1000/10Neg1000unk/twofoldwithunk/fold2/train.tsv"
# New_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_1000/10Neg1000unk/twofoldwithunk/fold2/dev.tsv"

# New_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_500/10Neg500unk/twofoldwithunk/fold1/train.tsv"
# New_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_ibm_150_5_10_500/10Neg500unk/twofoldwithunk/fold1/dev.tsv"

New_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_latex_text/PWC_latex_5_10_10000/twofoldwithunk/fold1/train.tsv"
New_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_latex_text/PWC_latex_5_10_10000/twofoldwithunk/fold1/dev.tsv"
# New_train_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_latex_text/PWC_latex_5_10_10000/twofoldwithunk/fold2/train.tsv"
# New_test_csv = "/nfs/home/kabenamualus/Research/task-dataset-metric-nli-extraction/data/pwc_latex_text/PWC_latex_5_10_10000/twofoldwithunk/fold2/dev.tsv"


# New_train_csv = IBM_train_csv
# New_test_csv = IBM_test_csv

In [53]:
train_New = pd.read_csv(New_train_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])
# train_IBM['label'] = train_IBM.label.apply(lambda x: "true" if x else "false")

test_New = pd.read_csv(New_test_csv, 
                    sep="\t", names=["label", "title", "TDM", "Context"])

In [54]:
len(train_New.drop_duplicates())

40538

In [55]:
len(train_New)

41733

In [60]:
2946+1262

4208

In [57]:
TDM_count = get_stats(train_New, test_New)

Train: 
Number of papers: 2951
Unknown count: 2370
Total leaderboards: 9853
Avg leaderboard per paper: 4.33
Distinc leaderboard: 1673
Distinct taks: 262
Distinc datasets: 853
Distinc metrics: 530
Max leaderboard per paper: 58
Min leaderboard per paper: 1
Least frequent leaderboard :[('Named Entity Recognition#CoNLL++#F1', 1), ('Graph Classification#HIV-fMRI-77#Accuracy', 1), ('Graph Classification#HIV-fMRI-77#F1', 1)]
Most frequent leaderboard :[('Object Detection#COCO test-dev#box AP', 55), ('Image Classification#CIFAR-10#Percentage correct', 55), ('Image Classification#ImageNet#Top 1 Accuracy', 73)]
 
 
Test: 
Number of papers: 1258
Unknown count: 981
Total leaderboards: 3856
Avg leaderboard per paper: 4.08
Distinc leaderboard: 1363
Distinct taks: 228
Distinc datasets: 712
Distinc metrics: 434
Max leaderboard per paper: 57
Min leaderboard per paper: 1
Least frequent leaderboard :[('Domain Adaptation#SVNH-to-MNIST#Accuracy', 1), ('Question Answering#SQuAD1.1 dev#F1', 1), ('Self-Superv

In [58]:
# TDM_count[:30]

In [59]:
print("Train")
print("======")
print(f"Number of papers: {round((2951 + 2942)/2)}")
print(f"Avg Unknown count: {round((2370 + 2348)/2)}")
print(f"Avg Total leaderboards: {round((9853 + 9374)/2)}")
print(f"Avg leaderboard per paper: {round((4.33 + 4.2)/2, 1)}")
print(f"Avg Distinc leaderboard: {round((1673 + 1663)/2)}")
print(f"Avg Distinct taks: {round((262 + 263)/2)}")
print(f"Avg Distinc datasets: {round((853 + 853)/2)}")
print(f"Avg Distinc metrics: {round((530 + 525)/2)}")
print(f"Unique Triples over all: {round((1724 + 1724)/2)}")

Train
Number of papers: 2946
Avg Unknown count: 2359
Avg Total leaderboards: 9614
Avg leaderboard per paper: 4.3
Avg Distinc leaderboard: 1668
Avg Distinct taks: 262
Avg Distinc datasets: 853
Avg Distinc metrics: 528
Unique Triples over all: 1724


In [44]:
print("Test")
print("======")
print(f"Number of papers: {round((1258 + 1267)/2)}")
print(f"Avg Unknown count: {round((981 + 1003)/2)}")
print(f"Avg Total leaderboards: {round((3856 + 4335)/2)}")
print(f"Avg leaderboard per paper: {round((4.08 + 4.37)/2, 1)}")
print(f"Avg Distinc leaderboard: {round((1363 + 1391)/2)}")
print(f"Avg Distinct taks: {round((228 + 229)/2)}")
print(f"Avg Distinc datasets: {round((712 + 717)/2)}")
print(f"Avg Distinc metrics: {round((434 + 435)/2)}")

Test
Number of papers: 1262
Avg Unknown count: 992
Avg Total leaderboards: 4096
Avg leaderboard per paper: 4.2
Avg Distinc leaderboard: 1377
Avg Distinct taks: 228
Avg Distinc datasets: 714
Avg Distinc metrics: 434


In [39]:
Unique Triples over all: 1850

Number of papers: 3753
Unknown count: 920
Total leaderboards: 11757
Avg leaderboard per paper: 4.15
Distinc leaderboard: 1820
Distinct taks: 291
Distinc datasets: 912
Distinc metrics: 553
Max leaderboard per paper: 58
Min leaderboard per paper: 1


In [22]:
# print("Train")
# print("======")
# print(f"Number of papers: {round((3753 + 3753)/2)}")
# print(f"Avg Unknown count: {round((923 + 920)/2)}")
# print(f"Avg Total leaderboards: {round((11690 + 11757)/2)}")
# print(f"Avg leaderboard per paper: {round((4.13 + 4.15)/2, 1)}")
# print(f"Avg Distinc leaderboard: {round((1791 + 1820)/2)}")
# print(f"Avg Distinct taks: {round((286 + 291)/2)}")
# print(f"Avg Distinc datasets: {round((905 + 912)/2)}")
# print(f"Avg Distinc metrics: {round((547 + 553)/2)}")
# print(f"Unique Triples over all: {round((1850 + 1850)/2)}")

Train
Number of papers: 3753
Avg Unknown count: 922
Avg Total leaderboards: 11724
Avg leaderboard per paper: 4.1
Avg Distinc leaderboard: 1806
Avg Distinct taks: 288
Avg Distinc datasets: 908
Avg Distinc metrics: 550
Unique Triples over all: 1850


In [18]:
# print("Test")
# print("======")
# print(f"Number of papers: {round((1608 + 1608)/2)}")
# print(f"Avg Unknown count: {round((378 + 381)/2)}")
# print(f"Avg Total leaderboards: {round((5094 + 5027)/2)}")
# print(f"Avg leaderboard per paper: {round((4.14 + 4.1)/2, 1)}")
# print(f"Avg Distinc leaderboard: {round((1556 + 1541)/2)}")
# print(f"Avg Distinct taks: {round((254 + 250)/2)}")
# print(f"Avg Distinc datasets: {round((806 + 790)/2)}")
# print(f"Avg Distinc metrics: {round((472 + 466)/2)}")

Test
Number of papers: 1608
Avg Unknown count: 380
Avg Total leaderboards: 5060
Avg leaderboard per paper: 4.1
Avg Distinc leaderboard: 1548
Avg Distinct taks: 252
Avg Distinc datasets: 798
Avg Distinc metrics: 469


In [17]:
count = []
for tdm in test_New.TDM.to_list():
    if tdm not in train_New.TDM.to_list():
        count.append(tdm)
print(count)

[]


In [18]:
count = []
for tdm in train_New.TDM.to_list():
    if tdm not in test_New.TDM.to_list():
        count.append(tdm)
print(len(count))

0


In [3]:
len(os.listdir("/nfs/home/kabenamualus/Research/task-dataset-metric-extraction/data/pdf_IBM"))

5363